In [23]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import pandas as pd
import numpy as np

import sys
sys.path.append('../tools/')

from fangraphs_projections import get_projections
import pre_process_script as pp
from league_data import process_league_data

In [25]:
%autoreload 2

id_dict = pd.read_csv('/Users/mitchv34/Work/fantasy_baseball_tools/data/SFBB Player ID Map - PLAYERIDMAP.csv')
id_dict = dict(zip( id_dict.FANTRAXID.to_list(), id_dict.IDFANGRAPHS.to_list() ))

# player_list = pd.read_csv("data/league/Fantrax-Players-Believers in BABIP (BIB).csv") # Old rosters
player_list = pd.read_csv("./data/league.csv") # New rosters
player_list.ID = player_list.ID.apply(lambda x: id_dict[x] if x in id_dict.keys() else np.nan)
player_list = player_list[player_list.ID == player_list.ID]
player_list.rename(columns={"ID": "playerid"}, inplace=True)
player_list = player_list.set_index("playerid")
player_list = player_list[["Player", "Team", "Position", "Status", "Salary", "ADP", "Age", "%D"]]
player_list.ADP = player_list.ADP.apply(lambda x: float(x) if x != "-" else 1000.0) 
# TODO: Change ADP to use Fangraphs projected ADP

In [35]:
names = ["steamer", "fangraphsdc", "zipsdc", "razzball", "atc", "thebat", "zips"]
# names = ['atc']
av_names = []   
for n in names:
    try:
        # Get projections from Fangraphs
        bats  = get_projections("all", "bat", n)
        pitch = get_projections("all", "pit", n)
        # Save data
        bats.to_csv(f"./data/{n}_bat.csv", index=False)
        pitch.to_csv(f"./data/{n}_pitch.csv", index=False)
        bats  = pp.pre_process_bat(bats).set_index("playerid")[["POINTS"]]
        player_list.to_csv("./data/league_data.csv")
        pitch = pp.pre_process_pitch(pitch).set_index("playerid")[["POINTS"]]
        ranks = pitch.add(bats, fill_value=0).sort_values("POINTS")
        player_list = player_list.join(ranks, how="inner")
        player_list.rename(columns={"POINTS":"POINTS_{}".format(n)}, inplace=True)
        av_names.append("POINTS_{}".format(n))
    except:
        print("{} data unavailable.".format(n))

# # Calculate totals
player_list["MEAN"] = np.mean(player_list[av_names], axis=1)
player_list["STD"] = np.std(player_list[av_names], axis=1)
player_list.sort_values("MEAN", inplace=True, ascending=False)
player_list["Rank"] = list(range(1, len(player_list)+1))
player_list["Rank_round"] = np.floor( (player_list.Rank  - 1 )/ 12 ) +  1
player_list["ADP_round"] = np.floor( (player_list.ADP  - 1 )/ 12 ) +  1
player_list["Keper_Value"] = player_list.Salary -  player_list.Rank_round
player_list["Draft_Value"] = player_list.ADP_round -  player_list.Rank_round

zipsdc data unavailable.
razzball data unavailable.
thebat data unavailable.
zips data unavailable.


In [28]:
player_list[["Rank", "Player"]].head(20)

,Rank,Player
playerid,,
18401,1,Ronald Acuna Jr.
20123,2,Juan Soto
5361,3,Freddie Freeman
19556,4,Yordan Alvarez
15640,5,Aaron Judge
19755,6,Shohei Ohtani
13611,7,Mookie Betts
18345,8,Kyle Tucker
27498,9,Spencer Strider


In [29]:
player_list[player_list.Position.str.contains("P")][["Player", "MEAN", "STD", "Rank"]].head(20)

,Player,MEAN,STD,Rank
playerid,,,,
19755,Shohei Ohtani,74.885255,0.0,6
27498,Spencer Strider,72.613145,0.0,9
13125,Gerrit Cole,70.609497,0.0,12
10310,Zack Wheeler,66.691536,0.0,16
14107,Kevin Gausman,66.515351,0.0,18
25436,George Kirby,66.510994,0.0,19
17085,Pablo Lopez,64.452084,0.0,23
16149,Aaron Nola,63.781979,0.0,24
19361,Corbin Burnes,63.644198,0.0,25


In [30]:
# CCount the number of "RP" on each team
player_list.groupby("Status")["Position"].apply(lambda x: x.str.contains("SP").sum())

Status
BBE          13
DN            6
Edgar         3
FA          123
JM            6
Mash         10
Patron#5      6
Pete          6
SS268         7
TCE           7
TYR           5
Tomy         11
sfernand      7
Name: Position, dtype: int64

In [31]:
# Top available players
available = player_list[(player_list.Status == "FA")
        # & ~(player_list.Position.str.contains("P"))
        #& (player_list.ADP < 200) 
            ].sort_values("Rank")#[["Player", "Position", "Rank", "ADP", "MEAN"]]

available.head(50)

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_atc,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,
13743,Max Fried,ATL,SP,FA,28,1000.0,29,-,58.054766,58.054766,0.0,52,5.0,84.0,23.0,79.0
19326,Bryan Reynolds,PIT,OF,FA,28,1000.0,28,-,56.109697,56.109697,0.0,66,6.0,84.0,22.0,78.0
19427,Shane Bieber,CLE,SP,FA,32,1000.0,28,-,53.169324,53.169324,0.0,88,8.0,84.0,24.0,76.0
3096,Kenley Jansen,BOS,RP,FA,31,1000.0,36,-,52.770844,52.770844,0.0,93,8.0,84.0,23.0,76.0
13074,Yu Darvish,SD,SP,FA,35,1000.0,37,-,52.418616,52.418616,0.0,98,9.0,84.0,26.0,75.0
11861,Jason Adam,TB,RP,FA,27,1000.0,32,-,50.772055,50.772055,0.0,110,10.0,84.0,17.0,74.0
19479,Aaron Civale,CLE/TB,SP,FA,7,1000.0,28,-,49.206054,49.206054,0.0,120,10.0,84.0,-3.0,74.0
17874,Nestor Cortes Jr.,NYY,SP,FA,20,1000.0,28,-,48.993254,48.993254,0.0,125,11.0,84.0,9.0,73.0
19343,A.J. Puk,MIA,RP,FA,18,1000.0,28,-,48.928939,48.928939,0.0,127,11.0,84.0,7.0,73.0


In [32]:
player_list[(player_list.Status == "TDRJ")].sort_values("MEAN", ascending=False).head(30)

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_atc,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,


In [33]:
player_list[ (player_list.ADP >= 85) & (player_list.ADP <= 120) ] 

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_atc,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,


In [33]:
player_list[(player_list.Rank_round <= 11)].sort_values("Draft_Value", ascending=False).head(13)

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
16578,Yandy Diaz,TB,"3B,INF",Edgar,24,235.53,31,92%,61.093659,58.539436,...,57.134660,54.584817,54.530640,56.864028,2.375314,56,5.0,20.0,19.0,15.0
20373,Andres Munoz,SEA,RP,TYR,13,236.22,24,94%,55.562149,55.256311,...,57.492625,55.608343,56.804627,55.963191,0.881122,67,6.0,20.0,7.0,14.0
13892,Paul Sewald,SEA,RP,DN,22,254.58,32,94%,52.024073,52.903662,...,53.401098,55.751540,51.530844,52.989732,1.372592,91,8.0,22.0,14.0,14.0
18568,Luis Arraez,MIA,"1B,2B,INF",sfernand,19,207.89,25,98%,55.582329,56.816902,...,55.397248,58.574768,60.805286,57.366215,1.860465,52,5.0,18.0,14.0,13.0
19569,David Bednar,PIT,RP,FAFO,24,209.36,28,97%,53.487404,55.208707,...,55.334512,57.126930,50.259723,54.717257,2.327939,79,7.0,18.0,17.0,11.0
21029,Jhoan Duran,MIN,RP,CR,1,218.82,25,96%,53.152773,53.369622,...,54.617743,52.469810,52.414141,53.064624,0.793858,90,8.0,19.0,-7.0,11.0
17027,Alex Verdugo,BOS,OF,Tomy,22,186.90,26,98%,52.440571,53.891278,...,52.038958,51.596721,60.483117,54.156301,3.005180,82,7.0,16.0,15.0,9.0
21132,Alexis Diaz,CIN,RP,Patron#5,20,224.42,26,97%,47.016670,49.990145,...,50.641600,53.473678,53.391789,51.123129,2.245896,115,10.0,19.0,10.0,9.0
14993,Scott Barlow,KC,RP,Pete,23,220.41,30,97%,49.452151,51.118444,...,51.369971,52.240998,53.047144,51.467757,1.102930,109,10.0,19.0,13.0,9.0


# Positional Rankings

In [34]:
player_list[(player_list.Position.str.contains("1B"))].sort_values("Rank").head(20)

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_atc,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,
5361,Freddie Freeman,LAD,"1B,INF",Mash,38,1000.0,34,-,80.294538,80.294538,0.0,3,1.0,84.0,37.0,83.0
14344,Matt Olson,ATL,"1B,INF",JM,37,1000.0,29,-,72.578392,72.578392,0.0,10,1.0,84.0,36.0,83.0
11579,Bryce Harper,PHI,"1B,INF",TCE,32,1000.0,31,-,71.769422,71.769422,0.0,11,1.0,84.0,31.0,83.0
19611,Vladimir Guerrero Jr.,TOR,"1B,INF",Mash,31,1000.0,24,-,65.853993,65.853993,0.0,20,2.0,84.0,29.0,82.0
16578,Yandy Diaz,TB,"1B,3B,INF",Edgar,30,1000.0,32,-,62.463214,62.463214,0.0,29,3.0,84.0,27.0,81.0
19251,Pete Alonso,NYM,"1B,INF",Pete,29,1000.0,28,-,61.318128,61.318128,0.0,35,3.0,84.0,26.0,81.0
18568,Luis Arraez,MIA,"1B,2B,INF",BBE,24,1000.0,26,-,59.890782,59.890782,0.0,42,4.0,84.0,20.0,80.0
9218,Paul Goldschmidt,STL,"1B,INF",JM,36,1000.0,36,-,58.881752,58.881752,0.0,47,4.0,84.0,32.0,80.0
20529,Nolan Jones,COL,"1B,INF,OF",sfernand,3,1000.0,25,-,57.724579,57.724579,0.0,54,5.0,84.0,-2.0,79.0


In [35]:
player_list[(player_list.Position.str.contains("2B"))].sort_values("Rank").head(20)

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
12533,Marcus Semien,TEX,"2B,INF",Patron#5,29,30.31,32,100%,56.086417,59.696243,...,57.671632,53.069663,69.190111,59.414876,5.030973,42,4.0,3.0,25.0,-1.0
18568,Luis Arraez,MIA,"1B,2B,INF",sfernand,19,207.89,25,98%,55.582329,56.816902,...,55.397248,58.574768,60.805286,57.366215,1.860465,52,5.0,18.0,14.0,13.0
16556,Ozzie Albies,ATL,"2B,INF",SS268,25,52.18,26,100%,50.131384,53.998107,...,53.317963,53.020824,59.323240,53.943729,2.730223,85,8.0,5.0,17.0,-3.0
13613,Ketel Marte,ARI,"2B,INF",Tomy,24,184.95,29,98%,51.518611,53.698263,...,51.435486,51.238611,57.466222,53.583757,2.450442,86,8.0,16.0,16.0,8.0
5417,Jose Altuve,HOU,"2B,INF",FAFO,29,119.68,32,100%,49.297436,51.202969,...,49.573999,48.448833,67.478343,52.813109,6.624557,94,8.0,10.0,21.0,2.0
15362,Jeff McNeil,NYM,"2B,INF,OF",FAFO,24,170.23,30,100%,51.948849,54.092672,...,51.918427,50.613406,54.083265,52.528100,1.240965,96,8.0,15.0,16.0,7.0
19470,Tommy Edman,STL,"2B,SS,INF",TYR,21,80.06,27,100%,48.352044,52.333986,...,51.722698,50.896728,60.342480,52.463298,3.736666,97,9.0,7.0,12.0,-2.0
19950,Andres Gimenez,CLE,"2B,INF",Edgar,11,91.70,24,100%,48.805006,52.658165,...,51.600088,50.511872,55.805944,51.743382,2.158455,106,9.0,8.0,2.0,-1.0
13301,Max Muncy,LAD,"2B,3B,INF",Pete,28,111.05,32,100%,48.990420,50.193492,...,54.374288,50.273089,54.397461,51.102864,2.411867,116,10.0,10.0,18.0,0.0


In [36]:
player_list[(player_list.Position.str.contains("3B"))].sort_values("Rank").head(20)


,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
13510,Jose Ramirez,CLE,"3B,INF",TYR,30,3.01,30,100%,72.265394,73.817891,...,76.181110,73.532836,76.808552,74.055524,1.873137,7,1.0,1.0,29.0,0.0
17350,Rafael Devers,BOS,"3B,INF",Tomy,28,20.33,26,100%,67.757872,71.216030,...,66.547359,66.943294,75.751150,69.902406,3.220393,15,2.0,2.0,26.0,0.0
17678,Alex Bregman,HOU,"3B,INF",uu,29,53.55,28,100%,65.567498,69.264967,...,63.912139,60.166572,70.509434,66.568529,3.731741,23,2.0,5.0,27.0,3.0
18360,Austin Riley,ATL,"3B,INF",TCE,22,21.16,25,100%,63.707937,66.112438,...,63.678763,64.242113,68.477917,65.207340,1.686495,25,3.0,2.0,19.0,-1.0
11493,Manny Machado,SD,"3B,INF",Patron#5,30,14.41,30,100%,63.643557,64.363707,...,67.801486,66.163115,65.671991,64.925639,1.889118,26,3.0,2.0,27.0,-1.0
9777,Nolan Arenado,STL,"3B,INF",Patron#5,29,32.67,31,100%,61.894308,63.467016,...,60.371868,63.105391,62.446587,62.083427,1.065309,33,3.0,3.0,26.0,0.0
25764,Bobby Witt Jr.,KC,"3B,SS,INF",DN,11,23.72,22,100%,55.895449,56.169262,...,57.783918,56.040219,62.398658,57.246651,2.432207,53,5.0,2.0,6.0,-3.0
16578,Yandy Diaz,TB,"3B,INF",Edgar,24,235.53,31,92%,61.093659,58.539436,...,57.134660,54.584817,54.530640,56.864028,2.375314,56,5.0,20.0,19.0,15.0
13301,Max Muncy,LAD,"2B,3B,INF",Pete,28,111.05,32,100%,48.990420,50.193492,...,54.374288,50.273089,54.397461,51.102864,2.411867,116,10.0,10.0,18.0,0.0


In [37]:
player_list[(player_list.Position.str.contains("SS"))].sort_values("Rank").head(20)

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
16252,Trea Turner,PHI,"SS,INF",Edgar,30,4.56,29,100%,64.699471,71.392723,...,72.921786,70.719888,78.123335,71.860909,3.984251,11,1.0,1.0,29.0,0.0
19709,Fernando Tatis Jr.,SD,"SS,INF,OF",TCE,23,24.69,24,100%,70.701203,69.264826,...,64.026522,65.041501,61.304568,65.843666,3.191237,24,2.0,2.0,21.0,0.0
19612,Bo Bichette,TOR,"SS,INF",SS268,25,21.61,25,100%,62.625310,62.666306,...,65.839608,64.195434,68.630831,64.070316,2.614263,28,3.0,2.0,22.0,-1.0
13624,Corey Seager,TEX,"SS,INF",TYR,28,45.35,28,100%,64.388726,61.096503,...,65.821292,61.997119,61.278572,61.899142,2.842908,35,3.0,4.0,25.0,1.0
23667,Wander Franco,TB,"SS,INF",Patron#5,17,83.10,22,100%,60.908287,60.756841,...,60.438357,59.973410,51.338795,58.634513,3.366686,45,4.0,7.0,13.0,3.0
14162,Carlos Correa,MIN,"SS,INF",FAFO,28,103.50,28,100%,57.786149,58.552859,...,57.482041,58.064481,61.468869,58.515861,1.361825,46,4.0,9.0,24.0,5.0
12161,Xander Bogaerts,SD,"SS,INF",SS268,29,76.45,30,100%,55.839708,56.442098,...,59.366664,59.727642,60.817298,57.891966,2.155936,51,5.0,7.0,24.0,2.0
25764,Bobby Witt Jr.,KC,"3B,SS,INF",DN,11,23.72,22,100%,55.895449,56.169262,...,57.783918,56.040219,62.398658,57.246651,2.432207,53,5.0,2.0,6.0,-3.0
12916,Francisco Lindor,NYM,"SS,INF",Tomy,30,36.48,29,100%,53.217517,55.976797,...,56.693232,56.104471,62.374983,56.679267,2.774621,58,5.0,3.0,25.0,-2.0


In [38]:
player_list[(player_list.Position.str.contains("OF"))].sort_values("Rank").head(60)

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
20123,Juan Soto,SD,OF,sfernand,25,6.64,24,100%,88.385991,84.217356,...,85.038777,83.252415,83.528776,83.872503,2.827398,2,1.0,1.0,24.0,0.0
15640,Aaron Judge,NYY,OF,uu,28,2.21,30,100%,81.578427,82.748481,...,81.720702,82.837946,84.762255,82.464632,1.196697,3,1.0,1.0,27.0,0.0
19556,Yordan Alvarez,HOU,OF,TYR,23,10.91,25,100%,72.307157,75.560835,...,74.721042,71.239403,82.437054,75.447812,3.603534,6,1.0,1.0,22.0,0.0
18345,Kyle Tucker,HOU,OF,uu,25,9.65,26,100%,78.142926,76.824526,...,73.090081,65.160487,74.118514,73.050716,4.240841,8,1.0,1.0,24.0,0.0
18401,Ronald Acuna Jr.,ATL,OF,Edgar,28,4.00,25,100%,76.082993,74.267730,...,73.094634,69.659460,73.154525,72.991679,2.000741,9,1.0,1.0,27.0,0.0
13611,Mookie Betts,LAD,OF,sfernand,30,10.90,30,100%,71.358958,70.947126,...,71.318258,69.972772,72.507680,70.912341,1.014435,13,2.0,1.0,28.0,-1.0
10155,Mike Trout,LAA,OF,uu,30,15.89,31,100%,69.340324,73.850441,...,70.823789,69.098307,63.178105,70.302420,3.942332,14,2.0,2.0,28.0,0.0
23697,Julio Rodriguez,SEA,OF,uu,1,5.87,22,100%,64.891379,66.280999,...,68.517441,67.283553,68.030649,66.621187,1.456696,22,2.0,1.0,-1.0,-1.0
19709,Fernando Tatis Jr.,SD,"SS,INF,OF",TCE,23,24.69,24,100%,70.701203,69.264826,...,64.026522,65.041501,61.304568,65.843666,3.191237,24,2.0,2.0,21.0,0.0


In [39]:
# SP
player_list[(player_list.Position.str.contains("SP")) & (~(player_list.Position.str.contains("RP")))].sort_values("Rank").head(60)


,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
19755,Shohei Ohtani,LAA,"UT,SP",uu,27,8.38,28,100%,130.445790,128.597540,...,133.815622,134.239918,135.161226,131.183120,3.641907,1,1.0,1.0,26.0,0.0
13125,Gerrit Cole,NYY,SP,SS268,30,12.40,32,100%,71.948656,71.855404,...,75.110939,69.547964,73.232936,72.082469,1.762380,10,1.0,1.0,29.0,0.0
10954,Jacob deGrom,TEX,SP,DN,30,29.68,34,100%,76.076777,75.398107,...,70.632751,71.983880,61.124707,71.657592,5.083546,12,1.0,3.0,29.0,2.0
19361,Corbin Burnes,MIL,SP,uu,23,11.71,28,100%,68.445391,68.810137,...,73.639801,67.097206,71.550904,69.657918,2.226998,16,2.0,1.0,21.0,-1.0
16149,Aaron Nola,PHI,SP,TYR,29,24.28,29,100%,66.334693,66.749607,...,71.081265,71.319313,71.634256,69.096007,2.278309,17,2.0,2.0,27.0,0.0
3137,Max Scherzer,NYM,SP,CR,30,37.57,38,100%,70.783471,69.368009,...,70.418333,67.791796,64.643157,68.567363,2.041542,18,2.0,4.0,28.0,2.0
8700,Justin Verlander,NYM,SP,Pete,30,36.02,40,100%,67.212963,66.725169,...,68.899955,67.913617,70.534499,68.205033,1.240738,19,2.0,3.0,28.0,1.0
19427,Shane Bieber,CLE,SP,DN,27,41.04,27,100%,62.016861,63.828243,...,64.831765,58.933659,67.059188,63.723882,2.646508,29,3.0,4.0,24.0,1.0
21483,Shane McClanahan,TB,SP,sfernand,18,34.52,25,100%,65.042189,63.952661,...,64.225271,61.197905,64.073932,63.399030,1.368091,30,3.0,3.0,15.0,0.0


In [40]:
# RP
player_list[(player_list.Position.str.contains("RP")) & (~(player_list.Position.str.contains("SP")))].sort_values("Rank").head(20)


,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
21032,Emmanuel Clase,CLE,RP,TYR,17,62.14,25,100%,59.618512,60.834925,...,63.274836,62.074124,64.085468,61.914772,1.478995,34,3.0,6.0,14.0,3.0
14212,Josh Hader,SD,RP,uu,27,77.61,28,100%,59.541839,61.743191,...,59.294238,58.963994,62.355237,60.910143,1.736252,38,4.0,7.0,23.0,3.0
7005,Ryan Pressly,HOU,RP,Patron#5,28,112.79,34,100%,54.464551,56.526186,...,56.192865,56.735991,56.488569,56.494729,1.192910,62,6.0,10.0,22.0,4.0
20666,Felix Bautista,BAL,RP,uu,28,148.97,27,100%,53.972107,56.041738,...,56.992081,55.787262,58.683165,56.398520,1.427517,63,6.0,13.0,22.0,7.0
20373,Andres Munoz,SEA,RP,TYR,13,236.22,24,94%,55.562149,55.256311,...,57.492625,55.608343,56.804627,55.963191,0.881122,67,6.0,20.0,7.0,14.0
18138,Ryan Helsley,STL,RP,uu,11,131.94,28,100%,56.407731,56.235764,...,55.617388,58.081994,49.595905,55.303160,2.671336,73,7.0,11.0,4.0,4.0
15816,Devin Williams,MIL,RP,DN,26,93.28,28,100%,52.403362,54.206398,...,55.932710,56.251806,56.486934,55.168390,1.436534,74,7.0,8.0,19.0,1.0
16122,Jordan Romano,TOR,RP,Pete,26,100.43,29,100%,54.144394,54.055656,...,56.615459,56.257766,54.086501,54.868354,1.114142,77,7.0,9.0,19.0,2.0
19569,David Bednar,PIT,RP,FAFO,24,209.36,28,97%,53.487404,55.208707,...,55.334512,57.126930,50.259723,54.717257,2.327939,79,7.0,18.0,17.0,11.0


In [41]:
player_list[player_list.Player.str.contains("CJ")].sort_values("Rank").head(20)

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
25768,CJ Abrams,WSH,"SS,INF",uu,8,221.89,22,95%,34.863134,38.269022,...,33.91787,31.906921,41.566513,36.394729,3.188113,437,37.0,19.0,-29.0,-18.0


In [42]:

player_list.sort_values("MEAN", inplace=True, ascending=False)

player_list = player_list[player_list.Rank_round <= 30]
player_list_keper = player_list.dropna()

In [43]:
player_list

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,
19755,Shohei Ohtani,LAA,"UT,SP",uu,27,8.38,28,100%,130.445790,128.597540,...,133.815622,134.239918,135.161226,131.183120,3.641907,1,1.0,1.0,26.0,0.0
20123,Juan Soto,SD,OF,sfernand,25,6.64,24,100%,88.385991,84.217356,...,85.038777,83.252415,83.528776,83.872503,2.827398,2,1.0,1.0,24.0,0.0
15640,Aaron Judge,NYY,OF,uu,28,2.21,30,100%,81.578427,82.748481,...,81.720702,82.837946,84.762255,82.464632,1.196697,3,1.0,1.0,27.0,0.0
5361,Freddie Freeman,LAD,"1B,INF",FAFO,30,13.91,33,100%,77.085391,78.808044,...,81.615794,78.052348,80.243817,78.761034,1.715683,4,1.0,2.0,29.0,1.0
19611,Vladimir Guerrero Jr.,TOR,"1B,INF",FAFO,25,12.91,24,100%,79.878568,74.760641,...,76.646933,78.667670,75.534852,75.737981,3.506502,5,1.0,1.0,24.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27465,Spencer Torkelson,DET,"1B,INF",uu,17,269.22,23,87%,35.365132,39.055795,...,36.202074,34.802114,46.776741,38.703853,4.075547,356,30.0,23.0,-13.0,-7.0
13621,Jeimer Candelario,WSH,"3B,INF",FA,1,342.88,29,49%,36.205239,39.031366,...,35.948550,34.745320,46.131448,38.685365,3.798946,357,30.0,29.0,-29.0,-1.0
16933,Elieser Hernandez,NYM,"SP,RP",FA,1,867.40,27,4%,40.518896,39.313730,...,34.118870,35.900045,44.292876,38.680010,3.275506,358,30.0,73.0,-29.0,43.0


In [44]:
player_list_keper.Status.value_counts()

FA          62
Tomy        28
sfernand    26
Pete        26
uu          25
FAFO        25
SS268       25
TCE         25
DN          24
Patron#5    24
TYR         23
CR          23
Edgar       22
W (Tue)      1
W (Thu)      1
Name: Status, dtype: int64

In [45]:
player_list_keper[player_list_keper.Status == "Canha Ge"].sort_values("Keper_Value", ascending=False).head(50)

,Player,Team,Position,Status,Salary,ADP,Age,%D,POINTS_steamer,POINTS_fangraphsdc,...,POINTS_atc,POINTS_thebat,POINTS_zips,MEAN,STD,Rank,Rank_round,ADP_round,Keper_Value,Draft_Value
playerid,,,,,,,,,,,,,,,,,,,,,


In [46]:
player_list.to_csv('data/proc/league_player_list_with_ranks.csv', index=True, index_label='IDFANGRAPHS')

In [47]:
rookies = ["23697", "26288", "sa3008130", "22810"]
team = player_list[player_list.Status == "TDRJ"]
team = team.loc[~team.index.isin(rookies)]
team["adj_sal"] = 100 * (31 - team.Salary) / 31 

In [48]:
import cvxpy

N = 11


utilities = np.array(team.MEAN - team.adj_sal) 

# The variable we are solving for
selection = cvxpy.Variable(len(utilities),  boolean = True)


# The sum of the weights should be less than or equal to P

number_constraint = sum( selection ) == N


# Our total utility is the sum of the item utilities
total_utility = utilities @ selection

# We tell cvxpy that we want to maximize total utility 
# subject to weight_constraint. All constraints in 
# cvxpy must be passed as a list
knapsack_problem = cvxpy.Problem(cvxpy.Maximize(total_utility), [number_constraint])

# Solving the problem
knapsack_problem.solve(solver=cvxpy.GLPK_MI)

team.loc[[ i == 1.0 for i in selection.value]]#, ["Player", "Position", "Salary", cost]]

ValueError: Invalid dimensions (0,).

In [ ]:
team.MEAN - team.adj_sal

playerid
19755    45.658202
15640    -4.419444
18345     0.561265
10155   -26.931699
19361     7.489677
14710   -25.979547
14212   -20.280090
12916   -37.034773
16578    20.731312
18138    51.769104
22250    48.130467
15890     8.198726
12861   -40.244058
20454    -1.916672
14161     0.841512
15474   -12.531355
20538    45.222970
17919    -9.678165
14524    43.295693
14527    41.378103
19447     0.712605
17479   -16.475028
19316    36.285319
10061    10.368600
dtype: float64

In [ ]:
team.MEAN / np.max(team.MEAN)

playerid
19755    1.000000
15640    0.638291
18345    0.552223
10155    0.539203
19361    0.531000
14710    0.496746
14212    0.466035
12916    0.436300
16578    0.433996
18138    0.424575
22250    0.396484
15890    0.387050
12861    0.386620
20454    0.383668
14161    0.380058
15474    0.376432
20538    0.374038
17919    0.373556
14524    0.359158
14527    0.344354
19447    0.329255
17479    0.321082
19316    0.305037
10061    0.304185
Name: MEAN, dtype: float64